# Neural Network Prediction of Sahelian Summer Rainfall
***

#### Resources:
* [Mardata Course](https://github.com/mardatade/Course-Python-for-Machine-Learning/blob/master/3.%20Neural%20Network.ipynb)
* [Keras for Data Scientists](https://keras.io/getting_started/intro_to_keras_for_engineers/#data-loading-amp-preprocessing)

#### Packages

In [1]:
import numpy as np
import pandas as pd 
import xarray as xr

from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as st

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.utils import shuffle
from sklearn import metrics


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard 
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import tensorflow_addons as tfa

from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

import dask
from dask import delayed
import dask.bag as db


#### Dask Client

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:35755")
client

Client Scheduler: tcp://127.0.0.1:35755 Dashboard: http://127.0.0.1:38657/status,Cluster Workers: 4 Cores: 16 Memory: 135.09 GB


In [3]:
# dask.config.set(scheduler='synchronous')

<br>
<br>

## 1. Data Loading & Preprocessing
***

<br>

### a) Loading & Normalization

**predictor:** contains the data used for the inputs  
**label:** from Sahelrainfall data serves as validation data

In [4]:
predictor = xr.open_dataset('data/da_pred_all.nc').to_dataframe()

predictor_unit = pd.DataFrame(
    data = StandardScaler().fit_transform(predictor), 
    columns = predictor.columns,
    index =  predictor.index
)


# load validatoin data (Summer Rainfall over Sahel and scale to [cm/month]) 
labels = np.mean(np.loadtxt("data/da_o_sahelprecip19012017.txt", skiprows=8,)[:,7:10] * 0.01,  axis=1)

labels = np.squeeze(StandardScaler().fit_transform(np.reshape(labels, (117,1))))


predictor_unit.head()

,siod_e,siod_w,sst_med,tsa,tna,sst_mdr,sata_lnh,sata_lsh,sata_onh,sata_osh,slp_darwin,slp_tahiti,amo,nao,pdo,np,nino12,nino3,nino34,nino4
year,,,,,,,,,,,,,,,,,,,,
1901,-1.100027,-1.152764,-0.745530,-0.595366,0.388372,0.608415,-0.123443,-0.732091,-0.497808,-0.737797,0.074807,1.634819,0.923204,0.917456,-0.193321,1.938388,-0.950168,-0.595561,-0.214314,-0.079270
1902,0.088643,0.340415,-1.507314,-0.954566,-0.346586,-0.173588,-1.289978,-0.201810,-1.175314,-0.987096,1.443896,2.682485,-0.620146,-1.172590,0.819716,-0.162154,0.991321,0.969845,1.099218,1.070532
1903,-0.900789,0.669332,-2.243639,-2.186294,-0.101970,0.283583,-1.333183,-1.076056,-1.415719,-1.333946,-0.071881,1.535042,-0.458290,-1.030410,-0.186187,0.530864,-0.371251,0.000784,0.524139,0.842095
1904,-0.949568,-1.056219,-0.079925,-1.975498,-2.214111,-1.894743,-1.135674,-1.133384,-1.863746,-1.778347,-0.903114,1.235708,-1.872482,1.447076,-0.892459,0.756497,-0.307712,-0.234313,-0.475713,-0.741738
1905,-0.034350,-0.632249,-0.718895,-1.684676,-1.334312,-1.014906,-1.314666,-0.595938,-1.284589,-0.954579,0.759351,-2.655622,-0.499163,-1.289888,0.545055,-0.326007,1.227830,1.497381,1.439037,1.032459


<br>

### b) PCA

In [5]:
# Scikit PCA transformation
pca = PCA()
principalComponents = pca.fit_transform(predictor_unit)


# Create Create Pandas DF from PCs
col = []
for i in range(1, 21):
    col.append(f'PC{i}')

predictor_pc = pd.DataFrame(
    data = principalComponents,
    columns = col,
    index =  predictor.index
)

predictor_pc = pd.DataFrame(
    data = StandardScaler().fit_transform(predictor_pc), 
    columns = predictor_pc.columns,
    index =  predictor_pc.index
)

# Test for unit-variance and zero mean:
# np.std(pred_pc)
# np.mean(pred_pc)
# pred_pc.head()

predictor_pc.head()

features = predictor_pc.to_numpy() #loc[:,['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', HPARAM['input_var_nine']]].to_numpy()


<br>
<br>

## 2. MODEL SETUP AND TUNING
***

<br>

### Build Model Function
---

In [6]:
def BuildModel(HPARAMS):
    model = keras.Sequential([
#         layers.Dropout(0.1, input_shape=(20,)),
        layers.Dense(HPARAMS['n_units'], 'relu' ,name="layer1", input_shape=(20,)),
        layers.Dropout(HPARAMS['dropout'], name='Dropout1'),
        
        layers.Dense(HPARAMS['n_units'], 'relu', name="layer2"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout2'),
        
        layers.Dense(HPARAMS['n_units'], 'relu', name="layer3"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout3'),
        
        layers.Dense(HPARAMS['n_units'], 'relu', name="layer4"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout4'),
        
        layers.Dense(HPARAMS['n_units'], 'relu', name="layer5"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout5'),
        
#         layers.Dense(HPARAMS['n_units'], 'relu', name="layer6"),
#         layers.Dropout(HPARAMS['dropout'], name='Dropout6'),
        
        layers.Dense(1, name='output'), #activation='linear'
    ])
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(
            learning_rate=HPARAMS['learn_rate']
        )
    )
    return model

testmodel = BuildModel({'optimizer': 'Adam', 'learn_rate': 0.1, 'n_units': 10, 'dropout':0.1})
print(testmodel.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 10)                210       
_________________________________________________________________
Dropout1 (Dropout)           (None, 10)                0         
_________________________________________________________________
layer2 (Dense)               (None, 10)                110       
_________________________________________________________________
Dropout2 (Dropout)           (None, 10)                0         
_________________________________________________________________
layer3 (Dense)               (None, 10)                110       
_________________________________________________________________
Dropout3 (Dropout)           (None, 10)                0         
_________________________________________________________________
layer4 (Dense)               (None, 10)                1

<br>

### Single Run Training & Error Calculation Funktion
---

In [7]:
def TrainModel(SPLIT, HPARAM, features):
    
    model = BuildModel(HPARAM)
    
    hist_dir = parent_dir + f"/run-{HPARAM['grid_num']:04d}" + f"/split-{SPLIT['split_num']:02d}"
    
    train_history = model.fit(
        features[SPLIT['train_index']],
        labels[SPLIT['train_index']],
        batch_size=HPARAM['batch_size'],
        epochs=200,
        verbose=0,
        callbacks=[earlystop],
    )
    

    y_train = np.squeeze(model.predict(features[SPLIT['train_index']]))
    y_test = np.squeeze(model.predict(features[SPLIT['test_index']]))


    train_error = y_train - labels[SPLIT['train_index']]
    train_mse = np.mean(train_error**2) 
    train_corr = st.pearsonr(y_train, labels[SPLIT['train_index']])[0]
    
    test_error = y_test - labels[SPLIT['test_index']]
    test_mse = np.mean(test_error**2) 
    test_corr = st.pearsonr(y_test, labels[SPLIT['test_index']])[0]

    training_length = len(train_history.history['loss'])
    
    metrics = {
        'train_mse': train_mse,
        'train_corr': train_corr,
        'test_mse': test_mse,
        'test_corr': test_corr,
        'training_length': training_length
    }
    
    return metrics

<br>

### Cross-Validation and Log Function
---

In [8]:
def TuneModel(HPARAM):
    
    
    with tf.summary.create_file_writer(parent_dir + f"/run-{HPARAM['grid_num']:04d}").as_default():
        hp.hparams({
           HP_LEARN_RATE: HPARAM['learn_rate'],
           HP_NUMBER_HIDDEN_UNITS: HPARAM['n_units'],
           HP_BATCH_SIZE: HPARAM['batch_size'],
           HP_DROPOUT: HPARAM['dropout']
        })
                
        metrics = SPLITS.map(lambda SPLIT: TrainModel(SPLIT, HPARAM, features)).compute()

        train_mse = [metric['train_mse'] for metric in metrics]
        train_corr = [metric['train_corr'] for metric in metrics]
        test_mse = [metric['test_mse'] for metric in metrics]
        test_corr = [metric['test_corr'] for metric in metrics]
        training_length = [metric['training_length'] for metric in metrics]
        
        result = {
        'train_mse_mu': np.mean(train_mse),
        'train_mse_sig': np.std(train_mse),
        'train_corr_mu': np.mean(train_corr),
        'train_corr_sig': np.std(train_corr),
        'test_mse_mu': np.mean(test_mse),
        'test_mse_sig': np.std(test_mse),
        'test_corr_mu': np.mean(test_corr),
        'test_corr_sig': np.std(test_corr), 
        'training_length_mu': np.mean(training_length),
        'training_length_sig': np.std(training_length),             
        }

        
        tf.summary.scalar(METRIC_TRAIN_MSE_MU,        result['train_mse_mu'],  step=1)
        tf.summary.scalar(METRIC_TRAIN_MSE_SIG,       result['train_mse_sig'],   step=1)
        tf.summary.scalar(METRIC_TRAIN_CORR_MU,       result['train_corr_mu'], step=1)
        tf.summary.scalar(METRIC_TRAIN_CORR_SIG,      result['train_corr_sig'],  step=1)
        tf.summary.scalar(METRIC_TEST_MSE_MU,         result['test_mse_mu'],   step=1)
        tf.summary.scalar(METRIC_TEST_MSE_SIG,        result['test_mse_sig'],    step=1)
        tf.summary.scalar(METRIC_TEST_CORR_MU,        result['test_corr_mu'],  step=1)
        tf.summary.scalar(METRIC_TEST_CORR_SIG,       result['test_corr_sig'],   step=1)
        tf.summary.scalar(METRIC_TRAINING_LENGTH_MU,  result['training_length_mu'],  step=1)
        tf.summary.scalar(METRIC_TRAINING_LENGTH_SIG, result['training_length_sig'],   step=1)
        
    return result

<br>

## Perform Random Search
---

<br>

#### Set Logs

##### Set Parent Directory

In [9]:
parent_dir = 'logs/rand1_2/'

##### Clear Directory

In [10]:
# rm -rf logs/rand1_1/*

<br>

### Hyperparameter Setup
***

In [11]:
###################################
#####EXAMPLE SETUP FOR TESTING#####
###################################


#GRID SERACH HYPERPARAMETER#
#---------------------------
HP_LEARN_RATE = hp.HParam('learn_rate', hp.RealInterval(0.0001, 1.0),display_name='Learning Rate') # LogScale
HP_NUMBER_HIDDEN_UNITS = hp.HParam('n_units', hp.IntInterval(10, 50),display_name='n Hidden Units') #uniform
HP_BATCH_SIZE = hp.HParam('batch_size', hp.IntInterval(1, 12),display_name='Batch Size') #Uniform
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.75),display_name='Dropout') #Uniform



#CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
#----------------------------------------------------
cv_param={
    'N_FOLDS': 10,         # number of folds -> small for Test Runs
#     'TEST_FRAC': .1    # factrion that is held out for test
}


#BAGGING PARAMETER (NO PART OF GRID SEARCH)#
#-------------------------------------------
# n_baggs = 5  # number of baggs -> small for test runs

earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)

# hist_dir = parent_dir + f"/run-{HPARAM['grid_num']:04d}" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=hist_dir)


####################
#####FULL SETUP#####
####################


# #GRID SERACH HYPERPARAMETER#
# #---------------------------
# HP_INPUT_VAR_NINE = hp.HParam('input_var_nine', hp.Discrete(['PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16']),display_name='9th Input Variable')
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['AdamW', 'SGDW']),display_name='Optimizer')
# HP_LEARN_RATE = hp.HParam('learn_rate', hp.Discrete([0.01, 0.1, 0.2]),display_name='Learning Rate')
# HP_WEIGHT_DECAY = hp.HParam('weight_decay', hp.Discrete([0.001, 0.01, 0.1]),display_name='Weight Decay')
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1, 3, 10, 30]),display_name='Batch Size')
# HP_EPOCHS = hp.HParam('n_epochs', hp.Discrete([30, 80, 120]),display_name='Epochs')


# #CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
# #----------------------------------------------------
# cv_param={
#     'N_FOLDS': 105,      # number of folds -> sample size as in Badr
#     'TEST_FRAC': .1    # factrion that is held out for test
# }


# #BAGGING PARAMETER (NO PART OF GRID SEARCH)#
# #-------------------------------------------
# n_baggs = 10 # number of baggs -> 10 as in Badr

<br>

### Metric Selection
---

In [12]:
METRIC_TRAIN_MSE_MU = 'train_mse_mu'
METRIC_TRAIN_MSE_SIG = 'train_mse_sig'
METRIC_TRAIN_CORR_MU = 'train_corr_mu'
METRIC_TRAIN_CORR_SIG = 'train_corr_sig'

METRIC_TEST_MSE_MU = 'test_mse_mu'
METRIC_TEST_MSE_SIG = 'test_mse_sig'
METRIC_TEST_CORR_MU = 'test_corr_mu'
METRIC_TEST_CORR_SIG = 'test_corr_sig'

METRIC_TRAINING_LENGTH_MU = 'training_length_mu'
METRIC_TRAINING_LENGTH_SIG = 'training_length_sig'
        

<br>

### Create HP Bag
***

In [13]:
nruns = 1000

learn_rate = 10**np.random.uniform(0,-4,nruns)
n_units = np.random.uniform(10,50,nruns).astype(int)
batch_size = np.random.uniform(1,12,nruns).astype(int)
dropout = np.random.uniform(0, 0.75, nruns)

hparams = []

for i in range(nruns):

    hparams.append(
            {
            'grid_num': i,
            'learn_rate': learn_rate[i],
            'n_units': n_units[i],
            'batch_size': batch_size[i],
            'dropout': dropout[i],       
            }
        )
                        
HPARAMS = db.from_sequence(hparams)
HPARAMS.take(1)

({'grid_num': 0,
  'learn_rate': 0.0046176244808819045,
  'n_units': 41,
  'batch_size': 6,
  'dropout': 0.10004230845861092},)

<br>

### Create Data Splits Bag (KFold with permutation)
***

In [14]:
split_num = 0
splits = []
for train, test in KFold(n_splits=cv_param['N_FOLDS'], shuffle=True).split(features):                      #KFold

    train = shuffle(train)
    test  = shuffle(test)
# for train, test in ShuffleSplit(n_splits=cv_param['N_FOLDS'], test_size=cv_param['TEST_FRAC']).split(predictor_pc):
    splits.append(
        {
        'train_index': train,
        'test_index': test,
        'split_num': split_num
        }
    )
    split_num += 1 
SPLITS = db.from_sequence (splits)
SPLITS.take(1, 2)

({'train_index': array([ 16,  32,  44, 110,  70,  75,   7,  60,  84, 106,  25,  58, 105,
          89,  39, 114,  53, 108,  78,  73,  86,  10,   1,  31,  92,  82,
          45,  64, 116,  62,  54,  74,  81,  41,  85,  63, 102, 113,  47,
          87,  94,   8,  14,  23,  71,  80,   4,  42,  67,  99,  17,  97,
          93,  91,  83,  72,  33, 109,  50,  28,  36,  18, 115,  49,  59,
          95,   5,  77,  48,   6,  51,  76,  38,  21,  12,  24,  37,  20,
          11,   3,  65,  26,  88, 100,  29,  40,  22,  15,   2,  30, 103,
          68,  55,   0, 107,  19,  66,  98,  43,  56, 104,  79,  96,  69,
          61]),
  'test_index': array([ 52,  57,  46,  90, 112,   9,  35,  27,  13,  34, 111, 101]),
  'split_num': 0},)

<br>

### Log Experiment Confiuration to TensorBoard
---

In [15]:
with tf.summary.create_file_writer(parent_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LEARN_RATE, HP_NUMBER_HIDDEN_UNITS, HP_BATCH_SIZE, HP_DROPOUT],
        metrics=[
            hp.Metric(METRIC_TRAIN_MSE_MU, display_name='Training Sample MSE µ'),
            hp.Metric(METRIC_TRAIN_MSE_SIG, display_name='Training Sample  MSE σ'),
            hp.Metric(METRIC_TRAIN_CORR_MU, display_name='Training Sample Correlation µ'),
            hp.Metric(METRIC_TRAIN_CORR_SIG, display_name='Training Sample  Correlation σ'),
            hp.Metric(METRIC_TEST_MSE_MU, display_name='Test Sample MSE µ'),
            hp.Metric(METRIC_TEST_MSE_SIG, display_name='Test Sample  MSE σ'),
            hp.Metric(METRIC_TEST_CORR_MU, display_name='Test Sample Correlation µ'),
            hp.Metric(METRIC_TEST_CORR_SIG, display_name='Test Sample  Correlation σ'),
            hp.Metric(METRIC_TRAINING_LENGTH_MU, display_name='Training Length µ'),
            hp.Metric(METRIC_TRAINING_LENGTH_SIG, display_name='Training Length σ'),
        ],
    )

<br>

### Run Model
***

In [16]:
%%time
results = HPARAMS.map(lambda HPARAM: TuneModel(HPARAM)).compute()

ValueError: array must not contain infs or NaNs

In [ ]:
results

# results

In [21]:
!kill 


<br>

### Close Client after finishing / before using it in another Notebook

In [ ]:
client.close()

<br>
<br>

***
***
<br>
<br>

<br>

### Bagging Function (Not used curretnly)
---

In [22]:
# def Bagging(hparams, features, model, train_index, test_index):
    
    
    
#     # set emty output matrices
#     y_train_bagging = np.zeros((train_index.size, n_baggs))
#     y_test_bagging = np.zeros((test_index.size, n_baggs))    
    
    
#     #Train the model 'n_baggs' times and store model predictions into matrice
#     for n in range(n_baggs):
        
# #         print ('baggin run', n)
# #         print ('PREDICTION ON TEST DATA:', y_test_bagging)
        
#         # Bootstrap sampling from training Data with Size(Training Data)
#         train_index_bootstrap = np.random.choice(train_index, train_index.size)

#         #Train the model 
#         model.fit(
#             features[train_index_bootstrap],
#             labels[train_index_bootstrap],
#             batch_size=hparams[HP_BATCH_SIZE],
#             epochs=hparams[HP_EPOCHS],
#             verbose=0
#         )
        
#         #Run the model for insample data and store in one matrix:
#         y_train_bagging[:, n] = np.squeeze(model.predict(features[train_index]))
        
#         # ... and for out of sample data        
#         y_test_bagging[:, n] = np.squeeze(model.predict(features[test_index]))

#     # return mean of the outputs over baggins (1st dimension)
#     return y_train_bagging.mean(1), y_test_bagging.mean(1)
# #     return y_test_bagging.mean(1)